In [1]:
import requests
from bs4 import BeautifulSoup
#importing beautiful soap for scrapping the data

## Dependencies

`BBeautifulSoup` :    pip install BeautifulSoup

In [12]:

import pandas as pd
import numpy as np
import itertools #to create efficent looping to fetch more data in a go
import re 
import random 

### Creating BS4 Functions for scrapping

In [27]:
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1980-01-01,2021-12-31&num_votes=50000,&count=200"
#
#https://www.imdb.com/search/title/?title_type=feature&release_date=2020-01-01,2021-12-31&num_votes=20000,&count=200 - 1000 Files done .. processing time 45 min can be used for unlablled
#movie released b/w 2021 and 2020 having votes more than 20k @sanjay 
#"https://www.imdb.com/search/title/?title_type=feature&release_date=1980-01-01,2021-12-31&num_votes=20000,&count=200"


#Fetching only 600 movie listing as the processing time is great. once we are good with code we can change the above filter.
def getSoup(url):
    """
    Utility function this get soup function will fetch the above url which stored in url var.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

def getReviews(soup):
    '''Function returns all reviews including postive and negative..'''
    
    # get a list of user ratings
    user_review_ratings = [tag.previous_element for tag in 
                           soup.find_all('span', attrs={'class': 'point-scale'})]        #can search div by inspect elementor
    
    
    # get the review tags
    user_review_list = soup.find_all('a', attrs={'class':'title'})
    ans = []
    for i in range(5):
        ans.append(user_review_list[random.randint(0, len(user_review_list) -1)])
    links = ["https://www.imdb.com" + tag['href'] for tag in ans]
    return links


def getReviewText(review_url):
    '''Returns the user review text given the review url.'''
    # get the review_url's soup
    soup = getSoup(review_url)
    # find div tags with class text show-more__control
    tag = soup.find('div', attrs={'class': 'text show-more__control'})
    return tag.getText()

def getMovieTitle(review_url):
    '''Returns the movie title from the review url.'''
    # get the review_url's soup
    soup = getSoup(review_url)
    # find h1 tag
    tag = soup.find('h1')
    return list(tag.children)[1].getText()

def getNounChunks(user_review):
    # create the doc object
    doc = nlp(user_review)
    # get a list of noun_chunks
    noun_chunks = list(doc.noun_chunks)
    # convert noun_chunks from span objects to strings, otherwise it won't pickle
    noun_chunks_strlist = [chunk.text for chunk in noun_chunks]
    return noun_chunks_strlist
movies_soup = getSoup(url)

In [28]:
movie_tags = movies_soup.find_all('a', attrs={'class': None})

# filter the a-tags to get just the titles
movie_tags = [tag.attrs['href'] for tag in movie_tags 
              if tag.attrs['href'].startswith('/title') & tag.attrs['href'].endswith('/')]

# remove duplicate links
movie_tags = list(dict.fromkeys(movie_tags))

print("There are a total of " + str(len(movie_tags)) + " movie titles")
print("Displaying 10 titles")
movie_tags[:10]

There are a total of 200 movie titles
Displaying 10 titles


['/title/tt2382320/',
 '/title/tt9421570/',
 '/title/tt1160419/',
 '/title/tt6264654/',
 '/title/tt3480822/',
 '/title/tt9376612/',
 '/title/tt2379713/',
 '/title/tt0381061/',
 '/title/tt3811906/',
 '/title/tt1270797/']

In [29]:
base_url = "https://www.imdb.com"
movie_links = [base_url + tag + 'reviews' for tag in movie_tags]
print("There are a total of " + str(len(movie_links)) + " movie user reviews")
print("Displaying 20 user reviews links")
movie_links[:20]

There are a total of 200 movie user reviews
Displaying 20 user reviews links


['https://www.imdb.com/title/tt2382320/reviews',
 'https://www.imdb.com/title/tt9421570/reviews',
 'https://www.imdb.com/title/tt1160419/reviews',
 'https://www.imdb.com/title/tt6264654/reviews',
 'https://www.imdb.com/title/tt3480822/reviews',
 'https://www.imdb.com/title/tt9376612/reviews',
 'https://www.imdb.com/title/tt2379713/reviews',
 'https://www.imdb.com/title/tt0381061/reviews',
 'https://www.imdb.com/title/tt3811906/reviews',
 'https://www.imdb.com/title/tt1270797/reviews',
 'https://www.imdb.com/title/tt1074638/reviews',
 'https://www.imdb.com/title/tt10954652/reviews',
 'https://www.imdb.com/title/tt0107120/reviews',
 'https://www.imdb.com/title/tt0087182/reviews',
 'https://www.imdb.com/title/tt6742252/reviews',
 'https://www.imdb.com/title/tt0830515/reviews',
 'https://www.imdb.com/title/tt6334354/reviews',
 'https://www.imdb.com/title/tt3228774/reviews',
 'https://www.imdb.com/title/tt9243804/reviews',
 'https://www.imdb.com/title/tt9620292/reviews']

In [30]:
movie_soups = [getSoup(link) for link in movie_links]

# get all 500 movie review links
movie_review_list = [getReviews(movie_soup) for movie_soup in movie_soups]

#movie_review_list = list(itertools.chain(*movie_review_list))
#print(len(movie_review_list))

#print("There are a total of " + str(len(movie_review_list)) + " individual movie reviews")
#print("Displaying 10 reviews")
#movie_review_list[:10]

In [18]:

movie_review_list = list(itertools.chain(*movie_review_list))
print(len(movie_review_list))

print("There are a total of " + str(len(movie_review_list)) + " individual movie reviews")
print("Displaying 10 reviews")
movie_review_list[:10]

1000
There are a total of 1000 individual movie reviews
Displaying 10 reviews


['https://www.imdb.com/review/rw7442041/',
 'https://www.imdb.com/review/rw7415763/',
 'https://www.imdb.com/review/rw7437362/',
 'https://www.imdb.com/review/rw7442432/',
 'https://www.imdb.com/review/rw7442041/',
 'https://www.imdb.com/review/rw7415054/',
 'https://www.imdb.com/review/rw7378607/',
 'https://www.imdb.com/review/rw7410656/',
 'https://www.imdb.com/review/rw7420451/',
 'https://www.imdb.com/review/rw7419538/',
 'https://www.imdb.com/review/rw7401641/',
 'https://www.imdb.com/review/rw7443754/',
 'https://www.imdb.com/review/rw7422998/',
 'https://www.imdb.com/review/rw7417279/',
 'https://www.imdb.com/review/rw7435254/',
 'https://www.imdb.com/review/rw7398834/',
 'https://www.imdb.com/review/rw7399600/',
 'https://www.imdb.com/review/rw7399600/',
 'https://www.imdb.com/review/rw7427080/',
 'https://www.imdb.com/review/rw7396415/',
 'https://www.imdb.com/review/rw7383859/',
 'https://www.imdb.com/review/rw7343751/',
 'https://www.imdb.com/review/rw7348382/',
 'https://w

In [19]:
review_texts = [getReviewText(url) for url in movie_review_list]

# get movie name from the review link
movie_titles = [getMovieTitle(url) for url in movie_review_list]

# label each review with negative or positive

# construct a dataframe
df = pd.DataFrame({
             'user_review': review_texts })

In [20]:
df.head(5) #converting it into data frama

,user_review
0,No time to die well is a must watch for Bond f...
1,Having revisited the previous Daniel Craig Bon...
2,"For the most part, I liked No Time to Die. How..."
3,STAR RATING: ***** Saturday Night **** Friday ...
4,No time to die well is a must watch for Bond f...


In [22]:
text_list = [m for m in df['user_review']]
#text_list

In [23]:
text_list_length = [len(m.split()) for m in text_list]     
df['length'] = text_list_length
df

,user_review,length
0,No time to die well is a must watch for Bond f...,246
1,Having revisited the previous Daniel Craig Bon...,360
2,"For the most part, I liked No Time to Die. How...",96
3,STAR RATING: ***** Saturday Night **** Friday ...,383
4,No time to die well is a must watch for Bond f...,246
...,...,...
995,My wife and I watched this at home on DVD from...,264
996,I can't imagine that anyone who read and liked...,230
997,I can't imagine that anyone who read and liked...,230
998,"When I saw the trailer for this movie, I was f...",1333


In [24]:
df = df[df['length'] < 250]
df

,user_review,length
0,No time to die well is a must watch for Bond f...,246
2,"For the most part, I liked No Time to Die. How...",96
4,No time to die well is a must watch for Bond f...,246
5,This film start with the news broadcasting abo...,157
6,L. A. is facing wild fires. Police officer Joe...,145
...,...,...
993,Stanley Kubrick's final film is far from the m...,154
994,"Kubrick is hard for me to get, and this film i...",184
996,I can't imagine that anyone who read and liked...,230
997,I can't imagine that anyone who read and liked...,230


In [25]:
#converting only reviews to CSV & removing the index
df.to_csv('data_scrapped/data.csv', index=False) 

In [26]:
import csv

with open("data_scrapped/data.csv", "r") as f:
        reader = csv.reader(f)
        rownumber = 0
        for row in reader:
             g=open(str(rownumber)+".txt","w")
             g.write(str(row))
             rownumber = rownumber + 1
             g.close()